<a href="https://colab.research.google.com/github/nahbos/AUT-Neural-Networks/blob/main/HW04/Q4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Copyright (C) 2022 Sobhan Moradian Daghigh**
######**Date: 5/5/2022**

In [38]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf, keras
import tensorflow_datasets as tfds
from keras.layers import Dense, Dropout, InputLayer, Conv2D, MaxPool2D, Flatten, AveragePooling2D, GlobalAveragePooling2D
from keras.utils.vis_utils import plot_model
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator


import warnings
warnings.filterwarnings("ignore")
import itertools
%load_ext tensorboard
import os

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [14]:
# import os
# import sys
# import glob
# import argparse
# import matplotlib.pyplot as plt

# from keras import __version__
# from keras.applications.inception_v3 import InceptionV3, preprocess_input
# from keras.layers import Dense, GlobalAveragePooling2D
# from keras.preprocessing.image import ImageDataGenerator
# from keras.optimizers import SGD

In [15]:
print(tf.__version__)

2.8.0


Load Drive

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
def drive_flusher():
  %rm -rf '/content/drive/MyDrive/ANN_HW04/logs'
  os.chdir('/content/drive/MyDrive/ANN_HW04')
  os.mkdir('./logs')

In [18]:
tb_callback = keras.callbacks.TensorBoard(log_dir='./ANN_HW04/logs', histogram_freq=1)

In [19]:
class EarlyStoppingCallback(keras.callbacks.Callback):
  def __init__(self, patience=0):
    super(EarlyStoppingCallback, self).__init__()
    self.patience = patience

  def on_train_begin(self, logs=None):
    self.best = np.inf
    self.wait = 0
  
  def on_epoch_end(self, epoch, logs=None):
    current_loss = logs.get('loss')
    if np.less(current_loss, self.best):
      self.best = current_loss
      self.wait = 0
      self.best_weights = self.model.get_weights()
    else:
      self.wait += 1
      if np.greater_equal(self.wait, self.patience):
        self.model.stop_training = True
        self.model.set_weights(self.best_weights)

In [20]:
es_callback = EarlyStoppingCallback(patience=5)

In [22]:
def load_data():
  datas = []
  sets = ['train', 'validation', 'test']
  for dset in sets:
    data = tfds.load('beans', split=dset, as_supervised=True)
    datas.append(data)
    print('  |_ {} loaded'.format(dset))

  return datas

In [28]:
datas = load_data()
train = datas[0]
validation = datas[1]
test = datas[2]

  |_ train loaded
  |_ validation loaded
  |_ test loaded


In [35]:
nb_train_samples = np.array([x[0] for x in train]).shape[0]
nb_val_samples = np.array([x[0] for x in validation]).shape[0]
nb_test_samples = np.array([x[0] for x in test]).shape[0]

In [36]:
def data_preprocess_and_generation(dataset, split, batch_size=32):

  preprocessed = dataset.map(lambda img, label: (tf.cast(img, tf.float32) / 255.0, label))

  # Generate data to avoid overfitting
  if split == 'train':
    features = np.array([x[0] for x in preprocessed])
    lables   = np.array([x[1] for x in preprocessed])
    train_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
      horizontal_flip=True,
      zoom_range = 0.2,
      rotation_range=20,
      fill_mode='reflect'
    )
    preprocessed = train_data_gen.flow(features, lables, batch_size=batch_size)

  else: 
    # Caches the elements in this dataset. loat it into the memory to go faster
    preprocessed = preprocessed.cache()
    preprocessed = preprocessed.batch(batch_size)
    preprocessed = preprocessed.prefetch(tf.data.AUTOTUNE)

  return preprocessed

In [37]:
batch_size = 64
train = data_preprocess_and_generation(train, 'train', batch_size=batch_size)
validation = data_preprocess_and_generation(validation, 'validation', batch_size=batch_size)
test = data_preprocess_and_generation(test, 'test', batch_size=batch_size)

In [42]:
IM_WIDTH, IM_HEIGHT = 299, 299 # Fixed size for InceptionV3
NB_EPOCHS = 5
BAT_SIZE = 64
FC_SIZE = 1024
NB_IV3_LAYERS_TO_FREEZE = 172

In [3]:
def setup_to_transfer_learn(model, base_model):
  """ Freeze all layers and compile the model """
  for layer in base_model.layers:
    layer.trainable = False
  model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
def add_new_last_layer(base_model, nb_classes):
  """Add last layer to the convnet
  Args:
    base_model: keras model excluding top
    nb_classes: # of classes
  Returns:
    new keras model with last layer
  """
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dense(FC_SIZE, activation='relu')(x) # new FC layer, random init
  predictions = Dense(nb_classes, activation='softmax')(x) # new softmax layer
  model = Model(input=base_model.input, output=predictions)
  return model

In [26]:
def setup_to_finetune(model):
  """Freeze the bottom NB_IV3_LAYERS and retrain the remaining top layers.
  note: NB_IV3_LAYERS corresponds to the top 2 inception blocks in the inceptionv3 arch
  Args:
    model: keras model
  """
  for layer in model.layers[:NB_IV3_LAYERS_TO_FREEZE]:
     layer.trainable = False
  for layer in model.layers[NB_IV3_LAYERS_TO_FREEZE:]:
     layer.trainable = True
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001 * int(batch_size / 32)), 
                        loss='sparse_categorical_crossentropy', 
                        metrics=['accuracy'])

In [43]:
def train():
  """Use transfer learning and fine-tuning to train a network on a new dataset"""
  nb_classes = 3
  nb_epoch = NB_EPOCHS

  # setup model
  base_model = InceptionV3(weights='imagenet', include_top=False) # include_top=False excludes final FC layer
  model = add_new_last_layer(base_model, nb_classes)

  # transfer learning
  setup_to_transfer_learn(model, base_model)

  history_tl = model.fit_generator(
    train,
    nb_epoch=nb_epoch,
    samples_per_epoch=nb_train_samples,
    validation_data=validation,
    nb_val_samples=nb_val_samples,
    class_weight='auto')

  # fine-tuning
  setup_to_finetune(model)

  history_ft = model.fit_generator(
    train,
    samples_per_epoch=nb_train_samples,
    nb_epoch=nb_epoch,
    validation_data=validation,
    nb_val_samples=nb_val_samples,
    class_weight='auto')

In [44]:
train()

87924736/87910968 [==============================] - 2s 0us/step


TypeError: ignored